In [2]:
import tensorflow as tf
print('TF version=',tf.__version__)

TF version= 1.10.0


In [1]:
import numpy as np
from keras.models import Sequential
from keras.layers.core import Dense, Activation, Dropout
from keras.layers.recurrent import LSTM, SimpleRNN
from keras.layers.wrappers import TimeDistributed

Using CNTK backend


In [2]:
# method for generating text
def generate_text(model, length, ix_to_char):
    vocab_size = len(ix_to_char)

    # starting with random character
    ix = [np.random.randint(vocab_size)]
    y_char = [ix_to_char[ix[-1]]]
    X = np.zeros((1, length, vocab_size))
    for i in range(length):
        # appending the last predicted character to sequence
        X[0, i, :][ix[-1]] = 1

        # magic
        ix = np.argmax(model.predict(X[:, :i + 1, :])[0], 1)
        y_char.append(ix_to_char[ix[-1]])

    gen_txt = ''.join(y_char)
    try:
        print(gen_txt)
    except:
        pass

    return gen_txt


# method for preparing the training data
def load_data(data_dir, seq_length):
    with open(data_dir, 'rb') as bin_data:
        data = bin_data.read().decode('utf8', 'ignore')
    # data = open(data_dir, 'r').read()
    chars = list(set(data))
    VOCAB_SIZE = len(chars)

    print('Data length: {} characters'.format(len(data)))
    print('Vocabulary size: {} characters'.format(VOCAB_SIZE))

    ix_to_char = {ix: char for ix, char in enumerate(chars)}
    char_to_ix = {char: ix for ix, char in enumerate(chars)}

    X = np.zeros((int(len(data) / seq_length), seq_length, VOCAB_SIZE))
    y = np.zeros((int(len(data) / seq_length), seq_length, VOCAB_SIZE))
    for i in range(0, int(len(data) / seq_length)):
        X_sequence = data[i * seq_length:(i + 1) * seq_length]
        X_sequence_ix = [char_to_ix[value] for value in X_sequence]
        input_sequence = np.zeros((seq_length, VOCAB_SIZE))
        for j in range(seq_length):
            input_sequence[j][X_sequence_ix[j]] = 1.
            X[i] = input_sequence

        y_sequence = data[i * seq_length + 1:(i + 1) * seq_length + 1]
        y_sequence_ix = [char_to_ix[value] for value in y_sequence]
        target_sequence = np.zeros((seq_length, VOCAB_SIZE))
        for j in range(seq_length):
            target_sequence[j][y_sequence_ix[j]] = 1.
            y[i] = target_sequence
    return X, y, ix_to_char


In [3]:
DATA = 'potter-nlp/final_data/allBooks.txt'
BATCH_SIZE = 50
LAYER_NUM = 2
HIDDEN_DIM = 500
SEQ_LENGTH = 50
WEIGHTS = ''           # filename of a trained model
GENERATE_LENGTH = 500
NB_EPOCH = 20
MODE = 'train'
EPI = 10        # epochs_per_iteration

### Model loading probably does not work

In [4]:
WEIGHTS='tensorflow/checkpoint_layer_2_hidden_500_epoch_390.hdf5'

In [4]:
WEIGHTS='cntk/checkpoint_layer_2_hidden_500_epoch_460.hdf5'

In [ ]:
backup = MODEL

In [5]:
# Creating training data
X, y, ix_to_char = load_data(DATA, SEQ_LENGTH)
VOCAB_SIZE = len(ix_to_char)


def create_model():
    # Creating and compiling the Network
    model = Sequential()
    model.add(LSTM(HIDDEN_DIM, input_shape=(None, VOCAB_SIZE), return_sequences=True))
    for i in range(LAYER_NUM - 1):
        model.add(LSTM(HIDDEN_DIM, return_sequences=True))
    model.add(TimeDistributed(Dense(VOCAB_SIZE)))
    model.add(Activation('softmax'))
    model.compile(loss="categorical_crossentropy", optimizer="rmsprop")
    return model


MODEL = create_model()


def initialize_model():
    if not WEIGHTS == '':
        # Loading the trained weights
        MODEL.load_weights(WEIGHTS)
        epoch = int(WEIGHTS[WEIGHTS.rfind('_') + 1:WEIGHTS.find('.')])
        print('Loaded weights for epoch {}'.format(epoch))
    else:
        epoch = 0
    return epoch


actual_epoch = initialize_model()

print('sample text (generated):')
# Generate some sample before training to know how bad it is!
generate_text(MODEL, GENERATE_LENGTH, ix_to_char)

if MODE == 'train' or WEIGHTS == '':
    while True:
        print('\n\nEpoch: {}\n'.format(actual_epoch))
        try:
            MODEL.fit(X, y, batch_size=BATCH_SIZE, verbose=1, epochs=EPI)
        except OSError:
            # sometimes an OSError occurs, especially on Windows 10 Systems...
            print('Encountered an Error while training this epoch')
            continue

        actual_epoch += EPI
        txt = generate_text(MODEL, GENERATE_LENGTH, ix_to_char)
        with open('generated_' + str(actual_epoch), 'wb') as out:
            out.write(txt.encode(errors='ignore'))
        MODEL.save_weights('checkpoint_layer_{}_hidden_{}_epoch_{}.hdf5'.format(LAYER_NUM, HIDDEN_DIM, actual_epoch))

# performing generation only
elif MODE == 'test':
    if WEIGHTS == '':
        print('Specify saved Weights!')
        exit(1)

    generate_text(MODEL, GENERATE_LENGTH, ix_to_char)
    print('\n\n')

else:
    print('\n\nNothing to do!')

Data length: 6164627 characters
Vocabulary size: 57 characters


/root/anaconda3/envs/cntk-py35/lib/python3.5/site-packages/keras/backend/cntk_backend.py:1116: UserWarning: Warning: CNTK backend does not support collapse of batch axis with inferred dimension. The reshape did not take place.
  'Warning: CNTK backend does not support '
/root/anaconda3/envs/cntk-py35/lib/python3.5/site-packages/cntk/core.py:361: UserWarning: your data is of type "float64", but your input variable (uid "Input4") expects "<class 'numpy.float32'>". Please convert your data beforehand to speed up training.
  (sample.dtype, var.uid, str(var.dtype)))


Loaded weights for epoch 460
sample text (generated):
’ba0)b6a9tab1'ba)tt,',)’b6ap:jb1'ba0,6bf),8]b:9’)bf”b817b:tdbj1aa)tb'p□□bf)9\b'1tb,ad bgv98‘ab,ab79,ab1pa5 b60)b69,’b98’b7,a0b9b□)9jb1'b01tt,f□)b09,t”bft178b1jj16,a)b098’b98’b’178b9b□,aa□)b6:,□)b98’ba1b09tt”‘6b)81t:1p6b)”)6dbg709ab’,’b”1pb69”y bt18b96\)’b96b0)tb)”)6b'□960)’b96ba01p]0ba0)”b7)t)b6pjj16)’ba1bf)bv1:'1ta9f□”b98’b017ba0)b8)7b6at)8]a0b')□□b6,□)8aba0)b]t)9ab09□□b'1□□17)’b0,:ba0t1p]0ba0)b9,tb98’b)s9:,8,8]ba0)b7,8’176b1'ba0)b□,ft9t”b□11\,8]b9ab09tt”b'1tb9b:1:)8ab1tba71b9jj9t)8a□”b0)b09’b81


Epoch: 460

Epoch 1/10


/root/anaconda3/envs/cntk-py35/lib/python3.5/site-packages/cntk/core.py:361: UserWarning: your data is of type "float64", but your input variable (uid "Input1087") expects "<class 'numpy.float32'>". Please convert your data beforehand to speed up training.
  (sample.dtype, var.uid, str(var.dtype)))


123292/123292 [==============================] - 102s 829us/step - loss: 3.3519
Epoch 2/10
123292/123292 [==============================] - 101s 815us/step - loss: 2.2994
Epoch 3/10
123292/123292 [==============================] - 101s 820us/step - loss: 2.0431
Epoch 4/10
123292/123292 [==============================] - 101s 821us/step - loss: 2.0003
Epoch 5/10
123292/123292 [==============================] - 103s 832us/step - loss: 1.9718
Epoch 6/10
123292/123292 [==============================] - 103s 832us/step - loss: 1.9510
Epoch 7/10
123292/123292 [==============================] - 102s 827us/step - loss: 1.9347
Epoch 8/10
123292/123292 [==============================] - 101s 822us/step - loss: 1.9213
Epoch 9/10
123292/123292 [==============================] - 106s 859us/step - loss: 1.9095
Epoch 10/10
123292/123292 [==============================] - 104s 843us/step - loss: 1.8995
ut of time to take to take to take to take to take to take to take to take to take to take to take t

SystemError: <built-in function Trainer_train_minibatch> returned a result with an error set

```bash
for file in $(ls generated* | sort -V); do echo $file; echo; cat $file; echo; echo; done
```